<a href="https://colab.research.google.com/github/gitHek/cakd5_2nd_project_2team/blob/develop/%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%80%EB%B9%84_%EC%9D%B4%ED%98%84%EC%A0%95_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Finetuning input Data 준비 코드 실습 

강의자 : 김재원

* 이 단계를 진행하기 위해서는 Finetuning을 위해 준비한 data.txt 파일이 필요합니다. 아직 data.txt 파일이 준비되지 않았다면, 먼저 데이터를 생성하고 진행해주세요.

* 다음 코드는 꼭 순서대로 실행시켜주셔야 tensorflow 버전 error가 나지 않습니다. 

* 코드를 실행시키기 전 제공해 드린 tokenizationK.py 코드와 vocab file "vocab.korean.rawtext.list" 를 최상위 경로에 업로드 시켜주세요.

In [ ]:
## downgrade tensorflow version
%tensorflow_version 1.x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/cakd5_colab')

In [ ]:
import os, re
import argparse

# get tokenizer python script
from tokenizationK import FullTokenizer

In [ ]:
## get tokenizer vocab file
tokenizer = FullTokenizer(vocab_file="/content/drive/MyDrive/cakd5_colab/vocab.korean.rawtext.list")

문제 1) "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?"와 같은 예시처럼
해당 데이터에서는 "/슬롯(레이블)명;엔티티/"의 형식으로 슬롯과 엔티티를 정리해 놨으므로,
이를 잡아 줄 수 있는 정규표현식을 준비한다.

* 힌트0. 정규표현식 recap
- https://colab.research.google.com/drive/1YH77oCCBLMP6yIUHf-BQozLCxWmRbdd7
  위의 경로에서 정규표현식 부분을 다시 보시고 작성하면 됩니다.

* 힌트1. slot_pattern
  - text1 = "/인물;한지민/" 
  - text2 = "/인물;한예슬/" 
  - text3 = "/가수;아이유/"
  위의 3개의 text를 모두 잡을 수 있는 정규표현식이다.
  기본적인 형식은 /OO;OO/ 이므로 OO에 들어가는 모든 문자를 가져오는 정규표현식을 찾아보면 된다.

* 힌트2. multi_spaces
  - multi_spaces는 말그대로 multiple한 space를 잡는 정규표현식이다. 참고로 정규표현식에서 space는 "\s"이다. 
  


In [ ]:
## ----------------- 문제 1 ---------------- ##
import re
slot_pattern = re.compile('/(.+?);(.+?)/')
multi_spaces = re.compile('\s+')
## ---------------------------------------- ##

In [ ]:
# slot_pattern = re.compile('/[가-힣]+;[가-힣]+/')도 가능

In [ ]:
sentence =  "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?"

In [ ]:
slot_pattern_found = slot_pattern.findall(sentence)
slot_pattern_found

[('인물', '한지민'), ('인물', '한예슬')]

In [ ]:
line_refined = slot_pattern.sub("/슬롯/", sentence)
line_refined

'/슬롯/과 /슬롯/ 나오는 드라마 있어?'

In [ ]:
entity = "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?"

In [ ]:
slot_index = 0
slot, entity = slot_pattern_found[slot_index]

In [ ]:
entity_tokens = " ".join(tokenizer.tokenize(entity))
tokens = ""
tags = ""
tokens += entity_tokens + " "
tags += (slot + " ") * len(entity_tokens.split())

In [ ]:
tokens

'한 지 민_ '

In [ ]:
print(type(tokens))

<class 'str'>


In [ ]:
len(tokens.split())

3

In [ ]:
tags

'인물 인물 인물 '

In [ ]:
len(tags.split())

3

In [ ]:
data = sentence.splitlines()
processed_data = [process_line(line, tokenizer) for line in data]
processed_data

[('한 지 민_ 과_ 한 예 슬 _ 나오는_ 드라마_ 있어 ?_', '인물 인물 인물 O 인물 인물 인물 인물 O O O O')]

In [ ]:
tokens = list(map(lambda x: x[0],processed_data))
tokens

['한 지 민_ 과_ 한 예 슬 _ 나오는_ 드라마_ 있어 ?_']

In [ ]:
#apply(lambda) 를 쓸 경우 str에는 apply가 안 된다며 에러가 남 -> map으로 수정

### process_file : seq.in, seq.out 생성
단방향 데이터가 있는 file_path를 argument로 주면 가공을 한 이후에 output_dir 아래의 2개의 파일(seq.in, seq.out)을 저장해 주는 함수 

문제 2) tokens와 tags를 각각 processed_data에서 list로 뽑기

* 힌트. process_file class는 process_line 함수를 사용한다.따라서 process_line 함수의 output을 출력해보고, process_data의 형식을 안다면, lambda 함수를 사용해 tokens와 tags를 list로 출력할 수 있다. 


In [ ]:
def process_file(file_path, output_dir):
  if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
  
  data = open(file_path).read().splitlines()

  ## line별로 processing
  processed_data = [process_line(line, tokenizer) for line in data]


  ## ----------------- 문제 2 ---------------- ##
  tokens = list(map(lambda x: x[0],processed_data))
  tags = list(map(lambda x: x[1],processed_data))
  ## ---------------------------------------- ##

  ## seq_in : 토큰들로만 이루어진 파일
  ## seq_out : 태그들로만 이루어진 파일
  seq_in = os.path.join(output_dir, "seq.in")
  seq_out = os.path.join(output_dir, "seq.out")

  with open(seq_in, "w") as f:
    f.write("\n".join(tokens)+ "\n")

  with open(seq_out, "w") as f:
    f.write("\n".join(tags)+ "\n")

### process_line : 데이터를 라인별로 처리해주는 함수
sentence를 input으로 받아 (token, tag)쌍으로 된 결과값을 반환한다.
예를 들어 "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?" 같은 input을 받게 되면,
        ('한 지민 과 한예 슬 나오 는 드라마 있 어 ?', '인물 인물 O 인물 인물 O O O O O O')와 같은 (토큰, 태그)쌍 반환

문제 3) 토큰의 개수와 슬롯의 개수가 맞지 않는다면 본래 라인과 더불어 토큰/슬롯들을 프린트해주는 코드를 작성하시오.

* 힌트. print문 안에 있는 내용을 살펴보고, 필요한 것들을 가져다가 if문으로 비교를 하시면 됩니다. 

In [ ]:
def process_line(sentence, tokenizer):
  slot_pattern_found = slot_pattern.findall(sentence)
  line_refined = slot_pattern.sub("/슬롯/", sentence)
  tokens = ""
  tags = ""
  slot_index = 0

  for word in line_refined.split():
    ## "/게임명;일곱개의 대죄/" --> ("게임명", "일곱개의 대죄")
    if word.startswith("/"):
      slot, entity = slot_pattern_found[slot_index]
      slot_index += 1

    ## 엔티티를 토크나이즈 한 후, 토큰별로 태그를 추가해준다. 
      entity_tokens = " ".join(tokenizer.tokenize(entity))

      tokens += entity_tokens + " "
      tags += (slot + " ") * len(entity_tokens.split())

      ## 조사가 붙은 것이며(eg. "/슬롯/이", "/슬롯/에서"),
    ## 조사에 대해서 추가적으로 토큰 및 태그를 추가해 준다.
      if not word.endswith("/"):
        ## 우선 "/" 뒤에 오는 조사를 찾아 준다.
        josa = word[word.rfind("/")+1:]
        josa_tokens = " ".join(tokenizer.tokenize(josa))

        tokens += josa_tokens + " "
        tags += "O " * len(josa_tokens.split())
    
    elif "/" in word:
      prefix = word.split("/")[0]
      tokenized_prefix = " ".join(tokenizer.tokenize(prefix))
      tokens += tokenized_prefix + " "
      tags += "O " * len(tokenized_prefix.split())

      slot, entity = slot_pattern_found[slot_index]
      slot_index += 1

      entity_tokens = " ".join(tokenizer.tokenize(entity))

      tokens += entity_tokens + " "
      tags += (slot + " ") * len(entity_tokens.split())

    else:
      word_tokens = " ".join(tokenizer.tokenize(word))
      tokens += word_tokens + " "
      tags += "O " * len(word_tokens.split())
  
  tokens = multi_spaces.sub(" ", tokens.strip())
  tags = multi_spaces.sub(" ", tags.strip())

  ## 만일 토큰의 개수와 슬롯의 개수가 맞지 않다면 본래 라인과 더불어 토큰/슬롯들을 프린트해준다.
  ## ----------------- 문제 3 ---------------- ##
  # [if 문으로 시작하는 코드 작성 ! ] 
  if len(tokens.split())!= len(tags.split()):

    print(sentence)
    print("\t" + tokens + "\t", len(tokens.split()))
    print("\t" + tags + "\t", len(tags.split()))
    print('\t'+slot+'\t',len(slot.split()))
  ## ---------------------------------------- ##
  
  return tokens, tags

In [ ]:
#len(tokens)!= len(tags) 로만 하면 길이의 차이가 발생, split()을 추가하면 길이 항상 동일 

In [ ]:
process_line( "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?", tokenizer)

('한 지 민_ 과_ 한 예 슬 _ 나오는_ 드라마_ 있어 ?_', '인물 인물 인물 O 인물 인물 인물 인물 O O O O')

### 주의사항

위의 코드를 참조해서 input/output argument를 받는 python script를 생성해주세요. python script의 이름은 prepare_data.py입니다. 
주의할 점은 tokenizer와 vocab file은 제공해드린 것을 사용해주세요.

* 아래의 주석 내용은 argparse의 기본적인 내용입니다. 참조해서 python 파일을 만들어주시면 됩니다.

* 또한 python script를 생성하고 나서는 tensorflow 버전과 python 버전을 잘 체크해주셔야 합니다. 

  - python version 3.7
  - tensorflow version 1.15

* 로컬이나 환경을 생성하셔서 위와 같이 버전을 맞춰주시고 코드를 돌리시면 무리없이 잘 돌아갈 것입니다. 

  - $ python prepare_data.py --input data.txt --output output

* 위와 같은 코드를 입력하셔서 python script를 돌리실 수 있습니다. 자세한 내용은 아래 argparse를 참조해주세요. 

* 수요일에는 prepare_data.py 정답 코드를 제공해드리겠습니다. 우선은 colab으로 함수의 구조를 익혀보시고 직접 script를 짜보시길 바랍니다. 궁금한 점이 있다면 카톡방에 물어봐주세요! 

In [ ]:
# -*- coding: utf-8 -*-

import os, re
import argparse
from tokenizationK import FullTokenizer

import pdb

############################################## TODO 경로 고치기 ###############################################
tokenizer = FullTokenizer(vocab_file="/content/drive/MyDrive/cakd5_colab/vocab.korean.rawtext.list")
###############################################################################################################

# "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?"와 같은 예시처럼
# 해당 데이터에서는 "/슬롯(레이블)명;엔티티/"의 형식으로 슬롯과 엔티티를 정리해 놨으므로,
# 이를 잡아 줄 수 있는 정규표현식을 준비한다.

slot_pattern = re.compile('/(.+?);(.+?)/')
multi_spaces = re.compile('\s+')


def process_file(file_path, output_dir):
    """
    단방향 데이터가 있는 file_path을 argument로 주면 가공을 한 이후에
    output_dir 아래에 2개의 파일(seq.in, seq.out)을 저장해 주는 함수.
    """
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    data = open(file_path).read().splitlines()

    # line별로 process를 해준 뒤,
    processed_data = [process_line(line, tokenizer) for line in data]
    ## tokens = [알맞은 코드를 써주세요]
    ## tags = [알맞은 코드를 써주세요]

    # seq_in : 토큰들로만 이루어진 파일
    # seq_out : 태그들로만 이루어진 파일
    seq_in = os.path.join(output_dir, "seq.in")
    seq_out = os.path.join(output_dir, "seq.out")

    with open(seq_in, "w") as f:
        f.write("\n".join(tokens)+ "\n")

    with open(seq_out, "w") as f:
        f.write("\n".join(tags)+ "\n")


def process_line(sentence, tokenizer):
    """
    데이터를 라인별로 처리해 주는 함수이다.
    라인을 주게 되면, (토큰, 슬롯)

    예를 들어 "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?" 같은 input을 받게 되면,
        ('한 지민 과 한예 슬 나오 는 드라마 있 어 ?', '인물 인물 O 인물 인물 O O O O O O')와 같은 (토큰, 태그)쌍으로 된 결과값을 반환한다.
    """
    slot_pattern_found = slot_pattern.findall(sentence)
    line_refined = slot_pattern.sub("/슬롯/", sentence)
    tokens = ""
    tags = ""
    slot_index = 0

    for word in line_refined.split():
        # "/게임명;일곱개의 대죄/" --> ("게임명", "일곱개의 대죄")
        if word.startswith("/"):
            slot, entity = slot_pattern_found[slot_index]
            slot_index += 1

            # 엔티티를 토크나이즈 한 후, 토큰별로 태그를 추가해 준다.
            entity_tokens = " ".join(tokenizer.tokenize(entity))

            tokens += entity_tokens + " "
            tags += (slot + " ") * len(entity_tokens.split())

            # 조사가 붙은 것이며(eg. "/슬롯/이", "/슬롯/에서"),
            # 조사에 대해서 추가적으로 토큰 및 태그를 추가해 준다.
            if not word.endswith("/"):
                # 우선 "/" 뒤에 오는 조사를 찾아 주고
                josa = word[word.rfind("/")+1:]
                josa_tokens = " ".join(tokenizer.tokenize(josa))

                tokens += josa_tokens + " "
                tags += "O " * len(josa_tokens.split())
            
        elif "/" in word:

            prefix = word.split("/")[0]
            tokenized_prefix = " ".join(tokenizer.tokenize(prefix))
            tokens += tokenized_prefix + " "
            tags += "O " * len(tokenized_prefix.split())

            slot, entity = slot_pattern_found[slot_index]
            slot_index += 1

            entity_tokens = " ".join(tokenizer.tokenize(entity))

            tokens += entity_tokens + " "
            tags += (slot + " ") * len(entity_tokens.split())

        else:
            word_tokens = " ".join(tokenizer.tokenize(word))

            tokens += word_tokens + " "
            tags += "O " * len(word_tokens.split())

    tokens = multi_spaces.sub(" ", tokens.strip())
    tags = multi_spaces.sub(" ", tags.strip())

    # 만일 토큰의 개수와 슬롯의 개수가 맞지 않다면 본래 라인과 더불어 토큰/슬롯들을 프린트해준다.
    if len(tokens.split())!= len(tags.split()):
        print(sentence)
        print("\t" + tokens + "\t", len(tokens.split()))
        print("\t" + tags + "\t", len(tags.split()))

    return tokens, tags


if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument("--input_dir", "-i", help = "Etri KorBERT dir", type = str, required=True)
    parser.add_argument("--output_dir", "-o", help = "output_dir", type = str, required=True)

   # [ i nput argument를 받는 코드를 추가해주세요]
   # [ ouput argument를 받는 코드를 추가해주세요]
    
    args = parser.parse_args()
    file_path = args.input
    output_dir = args.output

    process_file(file_path, output_dir)


usage: ipykernel_launcher.py [-h] --input_dir INPUT_DIR --output_dir
                             OUTPUT_DIR
ipykernel_launcher.py: error: the following arguments are required: --input_dir/-i, --output_dir/-o


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
pip list

In [ ]:
import tensorflow
tensorflow.__version__

'1.15.2'

In [ ]:
import argparse

# # # 인자값을 받을 수 있는 인스턴스 생성
# parser = argparse.ArgumentParser(description='사용법 테스트입니다.')

# # # 입력받을 인자값 등록
# parser.add_argument('--target', required=True, help='어느 것을 요구하냐')
# #옵션을 지정하지 않아도 되는 인수
# parser.add_argument('--env', required=False, default='dev', help='실행환경은 뭐냐')

# # # 입력받은 인자값을 args에 저장 (type: namespace)
# args = parser.parse_args()

# # # 입력받은 인자값 출력
# print(args.target)
# print(args.env)

In [ ]:
# !$ python prepare_data.py --input data.txt --output output
# usage: argparse_test.py [-h] --target TARGET [--env ENV]
# argparse_test.py: error: the following arguments are required: --target

# !$ python prepare_data.py --input data.txt --output output -h
# usage: argparse_test.py [-h] --target TARGET [--env ENV]

# # 사용법 테스트입니다.

# optional arguments:
#   -h, --help       show this help message and exit
#   --target TARGET  어느 것을 요구하냐
#   --env ENV        실행환경은 뭐냐